<a href="https://colab.research.google.com/github/brianshin12/19-lab/blob/master/Ch_4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.5 A2C 알고리즘 구현
## 4.5.1 테스트 환경  
테스트 환경은 OpenAI Gym에서 제공하는 'Pendulum-v0'이다. (https://gym.openai.com/envs/Pendulum-v0/)  
에이전트의 목표는 길이가 1인 진자를 위로 수직으로 세워서 오래 유지하는 것이다.에이전트가 측정할 수 있는 파라미터는 수직축 좌표, 수평축 좌표와 각속도이다.  
## 4.5.2 코드 개요  
전체적인 학습 프로세스를 구체적으로 살펴보겠다.  
> 1. 상태변수, 행동, 시간차 타깃, 어드밴티지($x_i, u_i, y_i, A_i)$를 초기화한다.  
batch_states, batch_action, batch_td_targets, batch_advantages = [  ], [  ], [  ], [  ]  

> 2. 환경을 초기화하고 환경으로부터 첫 번째 상태변수 $x_0$를 측정한다.  
state = self.env.reset()  

> 3. 액터 신경망을 이용해 행동 $u_0\sim\pi_\theta(u_0|x_0)$를 확률적으로 선택한다.  
action = self.actor.get_action(state)  

> 4. 행동이 범위 [-2, 2]를 벗어나지 않도록 제한한다.  
action = np.clip(action, -self.action_bound, self.action_bound)  

> 5. 행동 $u_0$를 실행해 보상 $r(x_0, u_0)$와 다음 상태변수 $x_1$을 얻는다. 여기서 done=1이면 에피소드가 종료되는 조건에 도달했음을 의미한다.  
next_state, reward, done, _ = self.env.step(action)  

> 6. Gym 환경과 학습 환경에서 사용하는 변수의 배열 모양이 다름을 고려하여 상태변수, 행동, 보상, 다음 상태변수의 배열 모양을 바꿔준다.  
state = np.reshape(state, [1, self.state_dim])  
next_state = np.reshape(next_state, [1, self.state_dim])  
action = np.reshape(action, [1, self.action_dim])  
reward = np.reshape(reward, [1, 1])  

> 7. 크리틱 신경망을 이용해 상태가치 $V_\phi(x_0)$와 다음 상태가치 $V_\phi(x_1)$을 계산한다.  
v_value = self.critic.model.predict(state)  
next_v_vale = self.critic.model.predict(next_state)  

> 8. 어드밴티지 $A_\phi(x_0, u_0)=r(x_0, u_0) + \gamma V_\phi(x_1) - V_\phi(x_0)$와 시간차 타깃 $y_0=r(x_0, u_0) + \gamma V_\phi(x_1)$을 계산한다. 여기서 학습용으로 사용할 보상의 범위를 [-16, 0]에서 [-1, 1]로 조정한다.  
train_reward = (reward+8)/8  
advantage, y_i = self.advantage_td_target(train_reward, v_value, next_v_value, done)  

> 9. 상태변수, 행동, 시간차 타깃, 어드밴티지 $(x_0, u_0, y_0, A_0)$를 배치에 저장한다.  
batch_state.append(state)  
batch_state.append(action)  
batch_state.append(y_i)  
batch_state.append(advantage)  

> 10. 배치가 N개 (BATCH_SIZE)만큼 쌓일 때까지는 학습하지 않고 저장만 한다.  
if len(batch_size) < self.BATCH_SIZE:
  state = next_state[0]  
  episode_reward += reward[0]  
  time += 1  
  continue  

> 11. 배치가 차면 각각 N개의 상태변수, 행동, 시간차 타깃, 어드밴티지$(x_i, u_i, y_i, A_i)_{i=1,\cdots ,N}$을 추출한다. 그리고 배치를 비운다.  
states = self.unpack_batch(batch_size)  
actions = self.unpack_batch(batch_action)  
td_targets = self.unpack_batch(batch_td_target)  
advantages = self.unpack_batch(batch_advantage)  
batch_states, batch_action, batch_td_targets, batch_advantages = [  ], [  ], [  ], [  ]  

> 12. 손실함수 $L= {1 \over 2N} \sum _i (y_i-V_\phi(x_i))^2$를 이용해 크리틱 신경망을 학습한다.  
self.critic.train_on_batch(states, td_targets)  

> 13. 목적함수 그래디언트 $\nabla_\theta J(\theta) \approx \nabla_\theta \sum_i(log \pi_\theta(u_i|x_i)A_\phi(x_i, u_i))$를 이용해 액터 신경망을 학습한다.  
self.actor.train(states, actions, advantages)  

> 14. 다시 상태변수 $x_i$를 이용해 행동 $u_i$를 계산하는 과정을 되풀이한다.  
state = next_state[0]  
episode_reward += reward[0]  
time += 1